이번 장에서 배우는 내용

- 첫 번째 신경망(neural network) 예제
- 텐서와 텐서에 사용할 수 있는 연산자들
- 신경망이 학습하는 방법(backpropagation과 gradient descent)

딥러닝을 이해하려면 텐서, 텐서 연산, 미분, 경사 하강법(gradient descent) 등등에 대한 이해가 필요함


# 2.1 A first look at a neural network

- 0~9까지 필기체 숫자를 분류하는 문제 해결 과정을 통해 신경망을 살펴 봄
- MNIST 데이터 세트를 사용, 0~9까지 총 10개의 범주로 나주어 지며 각 이미지는 grayscale이고 이미지의 크기는 28x28 임
- 데이터 세트는 60,000장의 학습 이미지, 10,000장의 테스트 이미지로 구성
- [추가 MNIST sample images]
- 기본 용어 설명
    - class : 분류 문제에서 범주(category)에 해당
    - samples: 데이터 세트에서 개별 데이터
    - label: 특정 sample과 연관 있는 class
- keras에서 MNIST 데이터 세트를 제공함

### Keras에서 MNIST 데이터 세트 읽어 오기

In [7]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print(f"train images shape: {train_images.shape}")
print(f"train labels shape: {train_labels.shape}")
print(f"image shape: {train_images[0].shape}")
print(f"type of image: {type(train_images[0])}")
print(f"type of label: {type(train_labels[0])}")

train images shape: (60000, 28, 28)
train labels shape: (60000,)
image shape: (28, 28)
type of image: <class 'numpy.ndarray'>
type of label: <class 'numpy.uint8'>


mnist 데이터 세트를 사용해서 신경망 학습시키고 추론하는 과정
- 학습 데이터를 인공 신경망에 입력
- 인공 신경망이 학습 데이터에서 label과 입력 이미지 사이의 관계(패턴)을 학습
- 테스트 이미지를 사용해서 학습 결과 확인(예측과 실제 값 사이의 정확도 계산)

**간단한 인공 신경망 구현하기**

In [8]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

2022-01-01 15:21:16.448909: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-01 15:21:16.449577: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-01 15:21:16.454547: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-01 15:21:16.455231: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-01 15:21:16.455843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

- 인공 신경망의 핵심 빌딩 블록은 레이어
- 여러 레이어를 쌓아 올려 네트워크 모델을 구성
- 레이어는 입력 데이터의 이면에 있는 데이터의 표현(특징, representations)을 검출 함
- 레이어를 거치면서 데이터를 표현하는 특징을 학습하게 됨. 이는 데이터 처리를 위해 체로 걸러내는 것과 유사함
- 위 예제로 사용하는 네트워크는 두 개의 dense layer로 구성되어 있으며 전연결 신경망(fully connected neural network)으로 구성
- 마지막 레이어는 10개의 범주를 분류하기 위해 softmax 레이어로 구성. 즉, 마지막 레이어의 출력은 10개의 범주에 속할 확률값인데, 각 범주에 속할 확률이 가장 큰 값에 속하는 범주로 판단할 때 사용하며 10개의 범주에 속할 확률값을 전부 더하면 1이 됨

모델을 학습하기 위해 세 가지가 더 필요함
- optimizer(최적화 방법): 학습 데이터에 기반해 성능을 개선하기 위해 모델 자신을 업데이트하는 방법
- loss function(손실 함수): 모델의 성능을 측정하는 방법
- metrics(평가 지표): 학습과 평가 과정에서 모델의 성능을 추적하고 관찰하기 위한 성능 지표

모델을 학습하기 위한 최적화 방법, 손실 함수, 평가 지표 설정 및 모델 생성

In [9]:
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

학습 데이터 전처리
- 모델에서 요구하는 입력 범위로 데이터 형식 몇 데이터 값 변환
- (60000, 28, 28) 크기에 uint8 형식의 [0, 255] 범위의 입력 데이터를 (60000, 28*28) 크기에 float32 형식의 [0, 1] 범위의 데이터로 변환

In [10]:
train_images = train_images.reshape((train_images.shape[0], train_images.shape[1] * train_images.shape[2]))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((test_images.shape[0], test_images.shape[1] * test_images.shape[2]))
test_images = test_images.astype("float32") / 255

모델 학습하기

In [11]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

2022-01-01 15:48:42.578422: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
469/469 [==============================] - 2s 2ms/step - loss: 0.2515 - accuracy: 0.9275
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.1026 - accuracy: 0.9700
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0682 - accuracy: 0.9796
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0498 - accuracy: 0.9846
Epoch 5/5
469/469 [==============================] - 1s 2ms/step - loss: 0.0379 - accuracy: 0.9886


학습 과정에서 loss와 accuracy가 표시됨. loss는 학습 과정에서 발생하는 정답(label)과 예측 결과 사이의 오차. accuracy는 학습 데이터에 대한 모델의 예측 정확도

학습이 완료된 모델을 사용해서 학습 과정에서 보지 못한 데이터를 사용해 모델의 예측 성능을 평가함

In [12]:
test_digits = test_images[0:10]
predictions = model.predict(test_digits)
predictions[0]

array([5.3305147e-09, 7.4112771e-10, 1.1226961e-06, 2.4484842e-05,
       1.1033354e-10, 6.7630630e-08, 1.1214364e-12, 9.9997330e-01,
       2.6655677e-08, 9.4304914e-07], dtype=float32)

`model.predict()`는 10개의 범주에 속할 확률을 반환 함. `predictions[0]`는 첫 번째 성능 평가 sample을 나타냄. `model.predict()`의 반환 결과는 크기가 10인 배열 임. 배열의 인덱스는 예측해야 하는 숫자 0부터 9를 의미하며 각 인덱스에 해당하는 값은 예측 결과가 해당 숫자일 확률을 의미함.

In [13]:
predictions[0].argmax()

7

`predictions[0].argmax()`를 호출하면 배열에서 값이 가장 큰 인덱스를 반환함. 결과 배열에서 값이 가장 큰 인덱스는 7이며 값은 9.9997330e-01 임. 즉, 첫 번째 평가 sample이 7일 확률이 99.99733% 임을 의미함

In [14]:
test_labels[0]

7

첫 번째 평가 데이터의 label을 확인하면 7임을 알 수 있으며 모델이 정확하게 예측한 것을 확인 할 수 있음

학습한 모델의 일반화 성능(학습 과정에서 보지 못한 데이터에 대한 예측 성능)은 어느 정도일까?

In [15]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"test_acc: {test_acc}")

313/313 [==============================] - 0s 992us/step - loss: 0.0635 - accuracy: 0.9810
test_acc: 0.9810000061988831


평가 세트에 대한 성능 평가 결과 98.1%의 정확도를 보임. 학습 과정에서의 정확도는 98.8%로 평가 데이터에 대한 성능과 약간의 차이가 있음. 학습 데이터와 평가 데이터 사이의 성능 차이가 과적합(overfitting)의 사례임. 학습 데이터에 대한 정확도는 높은데 평가 데이터에 대한 성능은 낮은 경우 학습 과정에서 보지 못한 데이터에 대해서 예측 성능이 낮다는 것은 분류 문제에서 분류 기준이 모호함을 의미함